In [ ]:



#include <opencv2/opencv.hpp>
#include "raspicam_cv.h"
#include <iostream>
#include <ctime>
#include <chrono>
#include <vector>
#include <algorithm>
#include <iterator>
#include <sstream>
#include <wiringPi.h>
#include <thread>

using namespace raspicam;
using namespace cv;
//using namespace std;

Mat frame, Matrix, framePers, frameGray,frameThresh, frameEdge, frameFinal;
Mat ROILane, frameFinalDuplicate, frameFinalDuplicate1, ROILaneEnd;
int LeftLanePos, RightLanePos, laneCenter, frameCenter, Result, laneEnd;

RaspiCam_Cv Camera;

std::stringstream ss;

std::vector<int> histrogramLane;
std::vector<int> histrogramLaneEnd;

Point2f Source[] = {Point2f(30,140),Point2f(370,140), Point2f(5,220), Point2f(395,220)};
Point2f Destination[] = {Point2f(100,0),Point2f(300,0), Point2f(100,240), Point2f(300,240)};

 void Setup ( int argc,char **argv, RaspiCam_Cv &Camera )
  {
    Camera.set ( CAP_PROP_FRAME_WIDTH,  ( "-w",argc,argv,400 ) ); // can be changed from 360 to 400
    Camera.set ( CAP_PROP_FRAME_HEIGHT,  ( "-h",argc,argv,240 ) );
    Camera.set ( CAP_PROP_BRIGHTNESS, ( "-br",argc,argv,50 ) );
    Camera.set ( CAP_PROP_CONTRAST ,( "-co",argc,argv,50 ) );
    Camera.set ( CAP_PROP_SATURATION,  ( "-sa",argc,argv,50 ) );
    Camera.set ( CAP_PROP_GAIN,  ( "-g",argc,argv ,50 ) );
    Camera.set ( CAP_PROP_FPS,  ( "-fps",argc,argv,0 ));

}

void Perspective() {
	line(frame,Source[0],Source[1],Scalar(0,0,255), 2);
	line(frame,Source[1],Source[3],Scalar(0,0,255), 2);
	line(frame,Source[3],Source[2],Scalar(0,0,255), 2);
	line(frame,Source[2],Source[0],Scalar(0,0,255), 2);
	
	Matrix = getPerspectiveTransform(Source, Destination);
	warpPerspective(frame,framePers, Matrix, Size(400,240));
}

void Capture () {
	Camera.grab();
	Camera.retrieve(frame);
	cvtColor(frame, frame, COLOR_BGR2RGB);
}

void Threshold() {
	cvtColor(framePers, frameGray, COLOR_RGB2GRAY);
	inRange(frameGray, 110, 255, frameThresh); // 2nd parameter @dark 50-120, indoor 120~150, outdoor 220~240
	// Canny Edge detection
	Canny(frameGray, frameEdge, 300,700, 3, false); // when bright environmnent, change this to 600,700 (@ dark 100,200)
	add(frameThresh,frameEdge,frameFinal);
	cvtColor(frameFinal,frameFinal,COLOR_GRAY2RGB);
	cvtColor(frameFinal, frameFinalDuplicate, COLOR_RGB2BGR); // used in histrogram function only
	cvtColor(frameFinal, frameFinalDuplicate1, COLOR_RGB2BGR); // used in histrogram function only
}

void Histrogram() {
	histrogramLane.resize(400);
	histrogramLane.clear();
	for(int i=0;i<400; ++i) {   // frame.size().width = 360
		ROILane = frameFinalDuplicate(Rect(i,140,1,100));  // Region of Interest Lane = ROILane
		divide(255,ROILane, ROILane);
		histrogramLane.push_back((int)(sum(ROILane)[0]));
	}
	
	histrogramLaneEnd.resize(400);
	histrogramLaneEnd.clear();
	for(int i=0;i<400; ++i) {   // frame.Size().width = 360
		ROILaneEnd = frameFinalDuplicate1(Rect(i,0,1,240));  // Region of Interest Lane = ROILane
		divide(255,ROILaneEnd, ROILaneEnd);
		histrogramLaneEnd.push_back((int)(sum(ROILaneEnd)[0]));
	}
	
	laneEnd = sum(histrogramLaneEnd)[0];
	std::cout<<"Lane End =  "<<laneEnd<<std::endl;
}

void LaneCenter();

void LaneFinder() {
	std::vector<int>::iterator LeftPtr;
	LeftPtr = std::max_element(histrogramLane.begin(), histrogramLane.begin()+180);
	LeftLanePos = std::distance(histrogramLane.begin(), LeftPtr);
	
	std::vector<int>::iterator RightPtr;
	RightPtr = std::max_element(histrogramLane.begin() + 220, histrogramLane.end());
	RightLanePos = std::distance( histrogramLane.begin(), RightPtr);
	
	line(frameFinal, Point2f(LeftLanePos, 0), Point2f(LeftLanePos,240), Scalar(0,255,0), 2);
	line(frameFinal, Point2f(RightLanePos, 0), Point2f(RightLanePos,240), Scalar(0,255,0), 2);
	
	Result = laneCenter - frameCenter;
}

void LaneCenter() {
		laneCenter = (RightLanePos - LeftLanePos)/2 + LeftLanePos+3;
		frameCenter = 400/2;
		line(frameFinal, Point2f(laneCenter,0), Point2f(laneCenter,240), Scalar(0,255,0), 3);
		line(frameFinal, Point2f(frameCenter,0), Point2f(frameCenter,240), Scalar(255,0,0), 3);
}

int main(int argc, char **argv) {
	
	wiringPiSetup();
	pinMode(21, OUTPUT);  // WPI 21 = BCM 5 = Nr.29 = inner 6th  (arduino const int D0 = 0;)
	pinMode(22, OUTPUT);  // WPI 22 = BCM 6 = Nr.31 = inner 5th (arduino const int D1 = 1;)
	pinMode(23, OUTPUT);  // WPI 23 = BCM 13 = Nr.33 = inner 4th (arduino const int D2 = 2;)
	pinMode(24, OUTPUT);  // WPI 24 = BCM 19 (MISO SPI1) = Nr. 35 = inner 3rd (arduino const int D3 = 3;)
	
	Setup(argc, argv, Camera);
	std::cout<<"Connecting to Camera"<<std::endl;
	if(!Camera.open()) {
		std::cout<<"Failed to connect\n";
		return -1;
	}
	
	std::cout<<"Camera ID = "<<Camera.getId()<<std::endl;
	while(1) {
		auto start = std::chrono::system_clock::now();
		
		Capture();
		Perspective();
		Threshold();
		Histrogram();
		LaneFinder();
		LaneCenter();
		
		if (laneEnd > 5000) {
			digitalWrite(21, 1);  // binary 0b0111 = 7 decimal
			digitalWrite(22, 1); 
			digitalWrite(23, 1);
			digitalWrite(24, 0);
			std::cout<<"Lane End\n";

		}
		
		// positive value, blue line is to the left from green
		// center of direction is to the left from road center
		// vehicle is too much to the left
		// vehicle must turn right 
		// right gears must decelerate
		// left gears must accelerate
		
		if(Result == 0) {
			digitalWrite(21, 0);  // binary 0b0000 = 0 decimal
			digitalWrite(22, 0); 
			digitalWrite(23, 0);
			digitalWrite(24, 0);
			std::cout<<"Forward\n";
		} else if(Result > 0 && Result < 10) {
			digitalWrite(21, 1);
			digitalWrite(22, 0);  // binary 0b0001 = 1 decimal
			digitalWrite(23, 0);
			digitalWrite(24, 0);
			std::cout<<"Right1\n";
		} else if(Result >= 10 && Result < 20) {
			digitalWrite(21, 0);
			digitalWrite(22, 1);  // binary 0b0010 = 2 decimal
			digitalWrite(23, 0);
			digitalWrite(24, 0);
			std::cout<<"Right2\n";
		} else if(Result >= 20 ) {
			digitalWrite(21, 1);
			digitalWrite(22, 1);  // binary 0b0011 = 3 decimal
			digitalWrite(23, 0);
			digitalWrite(24, 0);
			std::cout<<"Right3\n";
		}
		
		// negative value, blue line is to the right from green
		// center of direction is to the right from road center
		// vehicle is too much to the right
		// vehicle must turn left
		// right gears must accelerate
		// left gears must decelerate
		
		if(Result < 0 && Result > -10) {
			digitalWrite(21, 0);
			digitalWrite(22, 0);  // binary 0b0100 = 4 decimal
			digitalWrite(23, 1);
			digitalWrite(24, 0);
			std::cout<<"Left1\n";
		} else if(Result <= -10 && Result > -20) {
			digitalWrite(21, 1);
			digitalWrite(22, 0);  // binary 0b0101 = 5 decimal
			digitalWrite(23, 1);
			digitalWrite(24, 0);
			std::cout<<"Left2\n";
		} else if(Result <= -20 ) {
			digitalWrite(21, 0);
			digitalWrite(22, 1);  // binary 0b0110 = 6 decimal
			digitalWrite(23, 1);
			digitalWrite(24, 0);
			std::cout<<"Left3\n";
		}
		
		// writing the direction instruction in the camera captured image
		
		if(laneEnd > 5000) 
		{
			ss.str(" ");
			ss.clear();
			ss<<"Lane End ";
			putText(frame, ss.str(), Point2f(1,50), 0, 1, Scalar(255,0,0), 2);
		}
		
		else if(Result == 0) 
		{
			ss.str(" ");
			ss.clear();
			ss<<"Result = "<<Result<<" (Forward)";
			putText(frame, ss.str(), Point2f(1,50), 0, 1, Scalar(0,0,255), 2);
		}
		
		else if(Result > 0) 
		{
			ss.str(" ");
			ss.clear();
			ss<<"Result = "<<Result<<" (Move Right)";
			putText(frame, ss.str(), Point2f(1,50), 0, 1, Scalar(0,0,255), 2);
		}
		
		else if(Result < 0) 
		{
			ss.str(" ");
			ss.clear();
			ss<<"Result = "<<Result<<" (Move Left)";
			putText(frame, ss.str(), Point2f(1,50), 0, 1, Scalar(0,0,255), 2);
		}
		
		
		
		namedWindow("original", WINDOW_KEEPRATIO);
		moveWindow("original", 0,100);
		resizeWindow("original", 640, 480);
		imshow("original",frame);
		
		namedWindow("Perspective", WINDOW_KEEPRATIO);
		moveWindow("Perspective", 640,100);
		resizeWindow("Perspective", 640, 480);
		imshow("Perspective",framePers);

		
		namedWindow("Final", WINDOW_KEEPRATIO);
		moveWindow("Final", 1280,100);
		resizeWindow("Final", 640, 480);
		imshow("Final",frameFinal);
		
		// std::this_thread::sleep_for(std::chrono::milliseconds(10));
		waitKey(10);
		auto end = std::chrono::system_clock::now();
		std::chrono::duration<double> elapsed_seconds = end-start;
		float t = elapsed_seconds.count();
		int FPS = 1/t;
		std::cout<<"FPS = "<<FPS<<std::endl;
		
	}
	return 0;
}

// Region of interest

//0,0__________________0,360
//        /  |  \
//       /   |   \
//      /    |    \
//     /     |     \
//  0x/      |      \x1
//   /       |       \
//  /        |        \
// /         |         \
//x2____________________3x 240,360
//240,0



// How to find lanes
//   a  |                |
//<---->|      b         |
//<-----|--------------->|
//      |                |
// 0,140|                |
// .____|________________|____
// |    |                |    |
// |    |                |    |
// |    |                |    |
// |    |                |    |
// |____|________________|____|.360,240
// Rect(x1, y1, x2, y2)
// Rect(0, 140, 360, 240)
// then, the region of interest will be divided into stripes of 1 px, and each has a value 100x0 = 0 or 100x255 = 25500 (intensity), and saved in an array
// 0000010000000000000000100000


//int d = 42;
//int o = 052;
//int x = 0x2a;
//int X = 0X2A;
//int b = 0b101010; // C++14
//unsigned long long l1 = 18446744073709550592ull; // C++11
//unsigned long long l2 = 18'446'744'073'709'550'592llu; // C++14
//unsigned long long l3 = 1844'6744'0737'0955'0592uLL; // C++14
//unsigned long long l4 = 184467'440737'0'95505'92LLU; // C++14


Arduino uno

In [ ]:
const int EnableL = 5; // ~5 = PWM pin = pulse width modulation
const int HighL = 7; // changed from 6 to 7
const int LowL = 6;  // changed from 7 to 6

const int EnableR = 10; // ~10 = PWM
const int HighR = 8;
const int LowR = 9;

// _____________ 0% Duty Cycle
// -___-___-___- 25% Duty cycle = analogWrite(64)
// --__--__--__- 50% Duty Cycle = analogWrite(127)
// ---_---_---_- 75% Duty Cycle = analogWrite(191)
// ------------- 100% Duty Cycle = analogWrite(255)

const int D0 = 0;  // wpi 21 bcm 5  LSB = Least Significant Bit
const int D1 = 1;  // wpi 22 bcm 6
const int D2 = 2;  // wpi 23 bcm 13
const int D3 = 3;  // wpi 24 bcm 19 MSB = Most Significant Bit

int a,b,c,d,data;

void Data() {
  a = digitalRead(D0);
  b = digitalRead(D1);
  c = digitalRead(D2);
  d = digitalRead(D3);
  data = 8*d+4*c+2*b+a;
}

void setup() {
  // put your setup code here, to run once:
  pinMode(EnableL, OUTPUT);
  pinMode(HighL, OUTPUT);
  pinMode(LowL, OUTPUT);

  pinMode(EnableR, OUTPUT);
  pinMode(HighR, OUTPUT);
  pinMode(LowR, OUTPUT);

  pinMode(D0, INPUT_PULLUP);
  pinMode(D1, INPUT_PULLUP);
  pinMode(D2, INPUT_PULLUP);
  pinMode(D3, INPUT_PULLUP);

}
void Stop();
void Forward() {
  digitalWrite(HighL, LOW );
  digitalWrite(LowL, HIGH);
  analogWrite(EnableL, 255);

  digitalWrite(HighR, LOW );
  digitalWrite(LowR, HIGH);
  analogWrite(EnableR, 255);
  
}

void  Backward() {
  digitalWrite(HighL, HIGH );
  digitalWrite(LowL, LOW);
  analogWrite(EnableL, 255);

  digitalWrite(HighR, HIGH );
  digitalWrite(LowR, LOW);
  analogWrite(EnableR, 255);
  
}

// tested 255, 140, 80, 40

void Left1() {
  Stop(); delay(10);
  digitalWrite(HighL, LOW );
  digitalWrite(LowL, HIGH);
  analogWrite(EnableL, 140);

  digitalWrite(HighR, LOW );
  digitalWrite(LowR, HIGH);
  analogWrite(EnableR, 255);
}

void Left2() {
  Stop(); delay(10);
  digitalWrite(HighL, LOW );
  digitalWrite(LowL, HIGH);
  analogWrite(EnableL, 80);

  digitalWrite(HighR, LOW );
  digitalWrite(LowR, HIGH);
  analogWrite(EnableR, 255);
}

void Left3() {
  Stop(); delay(10);
  digitalWrite(HighL, LOW );
  digitalWrite(LowL, HIGH);
  analogWrite(EnableL, 40);

  digitalWrite(HighR, LOW );
  digitalWrite(LowR, HIGH);
  analogWrite(EnableR, 255);
}


void Right1() {
  Stop(); delay(10);
  digitalWrite(HighL, LOW );
  digitalWrite(LowL, HIGH);
  analogWrite(EnableL, 255);

  digitalWrite(HighR, LOW );
  digitalWrite(LowR, HIGH);
  analogWrite(EnableR, 140);
}

void Right2() {
  Stop(); delay(10);
  digitalWrite(HighL, LOW );
  digitalWrite(LowL, HIGH);
  analogWrite(EnableL, 255);

  digitalWrite(HighR, LOW );
  digitalWrite(LowR, HIGH);
  analogWrite(EnableR, 80);
}

void Right3() {
  Stop(); delay(10);
  digitalWrite(HighL, LOW );
  digitalWrite(LowL, HIGH);
  analogWrite(EnableL, 255);

  digitalWrite(HighR, LOW );
  digitalWrite(LowR, HIGH);
  analogWrite(EnableR, 40);  
}
void Stop() {
  digitalWrite(HighL, LOW );
  digitalWrite(LowL, HIGH);
  analogWrite(EnableL, 0);

  digitalWrite(HighR, LOW );
  digitalWrite(LowR, HIGH);
  analogWrite(EnableR, 0);
}

void UTurn() {
  analogWrite(EnableL,0);
  analogWrite(EnableR,0);
  delay(400);

  analogWrite(EnableL,255);
  analogWrite(EnableR,255);
  delay(1000);

  analogWrite(EnableL,0); // stabilize camera
  analogWrite(EnableR,0);
  delay(400);

  digitalWrite(HighL, HIGH ); // left
  digitalWrite(LowL, LOW);
  digitalWrite(HighR, LOW );
  digitalWrite(LowR, HIGH);
  analogWrite(EnableL,255);
  analogWrite(EnableR,255);
  delay(700);

  analogWrite(EnableL,0);
  analogWrite(EnableR,0);
  delay(400);

  digitalWrite(HighL, LOW ); // forward
  digitalWrite(LowL, HIGH);
  digitalWrite(HighR, LOW );
  digitalWrite(LowR, HIGH);
  analogWrite(EnableL,255);
  analogWrite(EnableR,255);
  delay(900);

  analogWrite(EnableL,0);
  analogWrite(EnableR,0);
  delay(400);

  digitalWrite(HighL, HIGH ); // left
  digitalWrite(LowL, LOW);
  digitalWrite(HighR, LOW );
  digitalWrite(LowR, HIGH);
  analogWrite(EnableL,255);
  analogWrite(EnableR,255);
  delay(700);

  analogWrite(EnableL,0);
  analogWrite(EnableR,0);
  delay(400);

  digitalWrite(HighL, LOW ); // forward
  digitalWrite(LowL, HIGH);
  digitalWrite(HighR, LOW );
  digitalWrite(LowR, HIGH);
  analogWrite(EnableL,150);
  analogWrite(EnableR,150);
  delay(300);
  
}

void loop() {
  Data();
  if (data == 0) {
    Forward();
  } else if ( data == 1) {
    Right1();
  } else if ( data == 2) {
    Right2();
  } else if ( data == 3) {
    Right3();
  } else if ( data == 4) {
    Left1();
  } else if ( data == 5) {
    Left2();
  } else if ( data == 6) {
    Left3();
  } else if (data == 7 ) {
    UTurn();
  } else if (data > 7 ) {
    Stop();
  }

  
}